In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import requests
import pandas as pd
import os

In [2]:
chrome_options = webdriver.ChromeOptions()
chrome_options.binary_location = os.environ.get("GOOGLE_CHROME_BIN")
chrome_options.add_argument("--headless")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--no-sandbox")
driver = webdriver.Chrome(executable_path=os.environ.get("CHROMEDRIVER_PATH"), chrome_options=chrome_options)

F:\kmkl\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: use options instead of chrome_options
  


TypeError: argument of type 'NoneType' is not iterable

In [ ]:
#webscarping stock market time series data from 1997 to today from unicredit website
#driver = webdriver.Chrome('C://Users//ADMIN//Downloads//chromedriver_win32//chromedriver.exe')
driver.get('https://1c-ir.mdgms.com/hist.html?lang=en')
year=driver.find_element_by_xpath("/html/body/div[1]/div/div/div[3]/div/div/div/div/div[1]/div[2]/div/form/div[3]/div[2]/div[3]/div")
year.click()
year1=driver.find_element_by_xpath("/html/body/div[1]/div/div/div[3]/div/div/div/div/div[1]/div[2]/div/form/div[3]/div[2]/div[3]/div/div/ul/li[24]")
year1.click()
data=driver.find_element_by_xpath("/html/body/div[1]/div/div/div[3]/div/div/div/div/div[1]/div[2]/div/form/div[5]/button")
data.click()
r= driver.current_url
excel = r.replace('HTML' , 'CSV')
data =pd.read_csv(excel, sep = ';', header = None)




# Forecasting of Closing Stock price for Unicredit by fitting automatic updating data into ARIMA from Unicredit.com

In [ ]:
#print(data)

In [ ]:
import os
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
from pylab import rcParams
rcParams['figure.figsize'] = 10, 6
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima_model import ARIMA
from pmdarima.arima import auto_arima
from sklearn.metrics import mean_squared_error, mean_absolute_error
import math
import numpy as np
import plotly.express as px

In [3]:
fig = px.line(data, x = 0, y = 4,labels={"4": "Closing Price", }, title='Uni Credit Closing Price Time Series')
fig.show()

NameError: name 'px' is not defined

In [ ]:
df=data[4]
df1=df.drop([0])
#print (df1)

date= data[[0,4]]
date1=date.drop([0])

#print (date1)

In [ ]:
k= pd.to_datetime(date1[0])
df1.index=k



# Testing Stationarity of the data of Closing Price

In [ ]:
#Test for staionarity
def test_stationarity(timeseries):
    #Determing rolling statistics
    rolmean = timeseries.rolling(12).mean()
    rolstd = timeseries.rolling(12).std()
    #Plot rolling statistics:
    #plt.plot(timeseries , color='blue',label='Original')
    
    plt.plot(rolmean, color='red', label='Rolling Mean')
    plt.plot(rolstd, color='black', label = 'Rolling Std')
    plt.legend(loc='best')
    plt.title('Rolling Mean and Standard Deviation')
    plt.show(block=False)
   
    print("Results of dickey fuller test")
    adft = adfuller(timeseries,autolag='AIC')
    # output for dft will give us without defining what the values are.
    #hence we manually write what values does it explains using a for loop
    output = pd.Series(adft[0:4],index=['Test Statistics','p-value','No. of lags used','Number of observations used'])
    for key,values in adft[4].items():
        output['critical value (%s)'%key] =  values
    print(output)
    
test_stationarity(df1)

# Trends and Seasonality in Data

In [ ]:
result = seasonal_decompose(df1, model='multiplicative', freq = 30)
fig = plt.figure()  
fig = result.plot()  
fig.set_size_inches(16, 9)

In [ ]:
d=df1.values


In [ ]:
fl = df1.astype(str).astype(float)



In [ ]:
l1 = np.log(fl)


In [ ]:
#df5= np.fromstring(df1,sep=',')
#df_log = np.log(df5)
#df6=pd.Series(df_log)
#df6.head
#print(df5)
#3print(df_log)

# Removing Trends and Seasonality by taking Log of the data

In [ ]:
from pylab import rcParams
rcParams['figure.figsize'] = 10, 6
#df_log = np.log(df5)
moving_avg = l1.rolling(12).mean()
std_dev = l1.rolling(12).std()
#plt.legend(loc='best')
plt.title('Moving Average')
plt.plot(std_dev, color ="black", label = "Standard Deviation")
plt.plot(moving_avg, color="red", label = "Mean")
plt.legend()
plt.show()

# Training data

In [ ]:
train_data, test_data = l1[3:int(len(l1)*0.9)], l1[int(len(l1)*0.9):]
plt.figure(figsize=(10,6))
plt.grid(True)
plt.xlabel('Dates')
plt.ylabel('Closing Prices')
plt.plot(l1, 'green', label='Train data')
plt.plot(test_data, 'blue', label='Test data')
plt.legend()

In [ ]:
model_autoARIMA = auto_arima(train_data, start_p=0, start_q=0,
                      test='adf',       # use adftest to find             optimal 'd'
                      max_p=3, max_q=3, # maximum p and q
                      m=1,              # frequency of series
                      d=None,           # let model determine 'd'
                      seasonal=False,   # No Seasonality
                      start_P=0, 
                      D=0, 
                      trace=True,
                      error_action='ignore',  
                      suppress_warnings=True, 
                      stepwise=True)
print(model_autoARIMA.summary())

In [ ]:
#model_autoARIMA.plot_diagnostics(figsize=(15,8))
#plt.show()

# Forecasting and Predcitons

In [ ]:
model = ARIMA(train_data.values, order=(3, 1, 2))  
fitted = model.fit(disp=-1)  
print(fitted.summary())

In [ ]:
# Forecast
fc, se, conf = fitted.forecast(590, alpha=0.05)  # 95% confidence
fc_series = pd.Series(fc, index=test_data.index)
lower_series = pd.Series(conf[:, 0], index=test_data.index)
upper_series = pd.Series(conf[:, 1], index=test_data.index)
plt.figure(figsize=(12,5), dpi=100)
plt.plot(train_data, label='training')
plt.plot(test_data, color = 'blue', label='Actual Stock Price')
plt.plot(fc_series, color = 'orange',label='Predicted Stock Price')
plt.fill_between(lower_series.index, lower_series, upper_series, 
                 color='k', alpha=.10)
plt.title('Uni Credit Stock Prediction')
plt.xlabel('Time')
plt.ylabel('Actual Stock Price')
plt.legend(loc='upper left', fontsize=8)
plt.show()

# Mean Squared Error,Absolute Error for final Predictons

In [ ]:
# report performance
mse = mean_squared_error(test_data, fc)
print('MSE: '+str(mse))
mae = mean_absolute_error(test_data, fc)
print('MAE: '+str(mae))
rmse = math.sqrt(mean_squared_error(test_data, fc))
print('RMSE: '+str(rmse))
mape = np.mean(np.abs(fc - test_data)/np.abs(test_data))
print('MAPE: '+str(mape))